In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
import json
import sys, getopt, pprint
import pymongo

In [2]:
url = 'https://www.eventbrite.com/d/malaysia--kuala-lumpur/all-events/'
listing = []

# loop through first 50 pages of the website
for i in range(1,50):
    crawl = requests.get(f"{url}{i}")
    soup = BeautifulSoup(crawl.text, "html.parser")
    print("Now scraping: Page",i)
    
    #save the scraped HTML to list
    listing.append(soup.find_all(class_="search-event-card eds-l-pad-hor-2 eds-l-mn-pad-hor-5 eds-l-pad-vert-3 eds-l-mn-pad-vert-5"))
    sleep(3)


Now scraping: Page 1
Now scraping: Page 2
Now scraping: Page 3
Now scraping: Page 4
Now scraping: Page 5
Now scraping: Page 6
Now scraping: Page 7
Now scraping: Page 8
Now scraping: Page 9
Now scraping: Page 10
Now scraping: Page 11
Now scraping: Page 12
Now scraping: Page 13
Now scraping: Page 14
Now scraping: Page 15
Now scraping: Page 16
Now scraping: Page 17
Now scraping: Page 18
Now scraping: Page 19
Now scraping: Page 20
Now scraping: Page 21
Now scraping: Page 22
Now scraping: Page 23
Now scraping: Page 24
Now scraping: Page 25
Now scraping: Page 26
Now scraping: Page 27
Now scraping: Page 28
Now scraping: Page 29
Now scraping: Page 30
Now scraping: Page 31
Now scraping: Page 32
Now scraping: Page 33
Now scraping: Page 34
Now scraping: Page 35
Now scraping: Page 36
Now scraping: Page 37
Now scraping: Page 38
Now scraping: Page 39
Now scraping: Page 40
Now scraping: Page 41
Now scraping: Page 42
Now scraping: Page 43
Now scraping: Page 44
Now scraping: Page 45
Now scraping: Page 

In [3]:
with open("eventbrite.csv", "w", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Event Name","Url", "Event Day", "Event Date", "Event Location", "Event Fees"])

    for ad in listing:
        for i in ad:
            name = str(i.find(class_="eds-event-card__formatted-name--is-clamped").get_text()).strip()
            url = i.find("a")["href"]
            day = str(i.find(class_="eds-text-bs--fixed eds-text-color--grey-600 eds-l-mar-top-1").get_text()).strip()
            day1= day[0:3]
            date = day[5:11]
            venue = i.find_all(class_="eds-text-bs--fixed eds-text-color--grey-600 eds-l-mar-top-1")[1].get_text()
            fee =  i.find_all(class_="eds-text-bs--fixed eds-text-color--grey-600 eds-l-mar-top-1")[2].get_text()

            writer.writerow([name, url, day1, date, venue, fee])

In [4]:
#Save in csv file to do monitor and cleaning
import pandas as pd

pd.read_csv("eventbrite.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1550 entries, 0 to 1549
Data columns (total 6 columns):
Event Name        1550 non-null object
Url               1550 non-null object
Event Day         1534 non-null object
Event Date        1534 non-null object
Event Location    1546 non-null object
Event Fees        1468 non-null object
dtypes: object(6)
memory usage: 72.7+ KB


In [5]:
#let's look on the scraped data

data = pd.read_csv("eventbrite.csv")
data.head()

,Event Name,Url,Event Day,Event Date,Event Location,Event Fees
0,Effective Safety & Health Committee(Buy 1 Get ...,https://www.eventbrite.com/e/effective-safety-...,Tue,Dec 10,"Novux Training Room, Kuala Lumpur, Wilayah Per...",Starts at RM839.92
1,Kelas Advance - SQD Level 1,https://www.eventbrite.com/e/kelas-advance-sqd...,Tue,Jul 30,"3 Towers @ KL City Centre, Ampang, Kuala Lumpur",Free
2,Kelas Advance - SQD Level 1,https://www.eventbrite.com/e/kelas-advance-sqd...,Tue,Aug 20,"3 Towers @ KL City Centre, Ampang, Kuala Lumpur",Free
3,Kelas Advance - SQD Level 1,https://www.eventbrite.com/e/kelas-advance-sqd...,Tue,"Aug 6,","3 Towers @ KL City Centre, Ampang, Kuala Lumpur",Free
4,Kelas Advance - SQD Level 1,https://www.eventbrite.com/e/kelas-advance-sqd...,Tue,Aug 27,"3 Towers @ KL City Centre, Ampang, Kuala Lumpur",Free


In [6]:
#cleaning data
data = data.drop_duplicates(subset="Url", keep='first').reset_index().drop(columns='index')
data = data.drop(['Url'], axis=1)

df = data.to_csv("eventbrite.csv")

In [7]:
datajson = data.to_json(orient='table')

In [8]:
with open('eventbrite.json', 'w') as json_file:
    json.dump(datajson, json_file)



In [ ]:
from pymongo import MongoClient

client=pymongo.MongoClient()
client = MongoClient('mongodb://localhost:27017')
db = client.db['eventbrite_db']
collection = db.collection['event']

df = pd.read_csv("eventbrite.csv") #csv file which you want to import
records_ = df.to_dict(orient = 'records')
result = db.collection.insert_many(records_ )

result.inserted_ids

Output data can be found and monitored on csv file or json file. Whilst the script can be found on eventbrite_website.py